En este notebook explicaremos como extraer los datos de las matrices de insumo-producto del reporte del 2016 de la Comisión Económica para América Latina y el Caribe (CEPAL).

Dicho reporte junto con las matrices, se puede encontrar en:
https://www.cepal.org/es/publicaciones/40271-la-matriz-insumo-producto-america-sur-principales-supuestos-consideraciones

# Extracción y limpieza de matrices de insumo producto

Expondremos dos opciones, uno directo de la página web  y otro por medio del repositorio local

In [1]:
import pandas as pd

paises = ['Argentina', 'Bolivia', 'Brasil', 'Chile', 'Colombia', 'Ecuador',\
          'Paraguay', 'Peru', 'Uruguay', 'Venezuela']

paises_region = ['Argentina', 'Bolivia', 'Brasil', 'Chile', 'Colombia', 'Ecuador',\
          'Paraguay', 'Peru', 'Uruguay', 'Venezuela', 'Sudamerica']

### Opción 1 - Directo del repositorio de la CEPAL

In [2]:
dir_repositorio = 'https://repositorio.cepal.org/bitstream/handle/11362/40271/'
#La dirección del repositorio en donde estan guardados los archivos de la Cepal
matrices_IP = dict()

#Agregando solo a los países
for i, pais in enumerate(paises, start=2):
    
    #Cada archivo esta guardado tanto por un numero como por pais
    matrices_IP[pais] = pd.read_excel(dir_repositorio+str(i)+'/MIP2005'+pais+'.xlsx'
                         ,skiprows=[0], usecols=range(1,48))
    
    #Limpiamos los titulos de las columnas y las filas
    matrices_IP[pais].columns = [x.strip() for x in matrices_IP[pais].columns]
    matrices_IP[pais]['Clasificación 40'] = [x.strip() for x in matrices_IP[pais]['Clasificación 40']]
    matrices_IP[pais].set_index('Clasificación 40', inplace=True)
    matrices_IP[pais].index.rename('Sector', inplace = True)

### Opcion 2 - Usando el directorio local

In [3]:
import glob

In [ ]:
#Introduzca la dirección donde está los notebooks
dir_local = sorted(glob.glob(
    ' Introduzca aquí la dirección/.../01 - Datos y limpieza/00 - Datos de la CEPAL/*.xlsx'))

#Dir_local es una lista con todos los archivos con terminación .xlsx
# y que están en la carpeta de 00 - Datos
matrices_IP = dict()

for pais, direccion in zip(paises, dir_local):
    
    matrices_IP[pais] = pd.read_excel(direccion ,skiprows=[0], usecols=range(1,48))
    
    matrices_IP[pais].columns = [x.strip() for x in matrices_IP[pais].columns]
    matrices_IP[pais]['Clasificación 40'] = [x.strip() for x in matrices_IP[pais]['Clasificación 40']]
    matrices_IP[pais].set_index('Clasificación 40', inplace=True)
    matrices_IP[pais].index.rename('Sector', inplace = True)

In [5]:
#Para cualquiera de los dos casos se obtiene una matriz como esta
matrices_IP['Chile']

,Agricultura y forestal,Caza y pesca,Minería (energía),Minería (no energía),Carne y derivados,"Molinería, panadería y pastas",Azúcar y productos de confitería,Otros productos alimenticios,Bebidas,Productos de tabaco,...,Correo y telecomunicaciones,Finanzas y seguros,Servicios a empresas de todo tipo,Otros servicios,DI,X,Consumo,FBK,Ajuste Preliminar,DT a PB
Sector,,,,,,,,,,,,,,,,,,,,,
Agricultura y forestal,1532.859447,3.053670,1.765685,0.085841,804.614593,584.170803,30.695211,1456.080999,469.249094,17.469883,...,0.000000,0.000000,0.445048,11.176361,5828.832801,2513.444732,2444.239394,283.909061,12.908884,11083.334873
Caza y pesca,0.364597,33.987735,0.002138,0.005380,0.026362,0.148074,0.000009,574.654970,0.004003,0.000003,...,0.000410,0.000058,0.002936,1.744350,654.718368,1807.300587,311.246882,6.942412,5.003719,2785.211967
Minería (energía),29.292457,6.461051,33.736801,132.378800,21.258668,85.688891,26.829959,535.197897,70.851966,0.498879,...,31.671687,20.758658,258.682532,28.135920,7377.345574,226.933712,231.821362,207.088594,2333.270723,10376.459964
Minería (no energía),81.145831,4.881360,59.524052,945.391431,0.587266,15.883211,0.572135,57.251276,4.384891,0.937951,...,3.384287,2.394635,75.104264,5.403199,3083.576328,19059.724952,0.000000,-116.739549,42.861885,22069.423617
Carne y derivados,7.907584,19.123497,0.815742,1.348692,192.234925,16.013365,0.004954,60.742340,0.564490,0.048878,...,0.619923,0.269507,3.716434,46.826784,484.173033,335.269408,938.308894,-14.414354,-9.375786,1733.961195
"Molinería, panadería y pastas",74.179370,7.253920,3.908494,1.518394,0.961146,704.652205,0.098398,91.857277,43.785142,0.085186,...,0.764610,0.304989,5.091863,23.428605,1035.955180,607.137912,2007.641675,-30.841504,90.143075,3710.036338
Azúcar y productos de confitería,3.728347,0.011562,0.036002,0.044929,0.004650,2.522372,0.106215,9.997741,10.449587,0.006781,...,0.086888,0.089719,0.943844,0.435183,30.796001,20.273698,59.681476,-0.916830,0.451741,110.286086
Otros productos alimenticios,1019.015138,649.692170,25.239489,5.960461,7.359377,128.609465,0.026236,712.031352,180.390656,0.170124,...,1.327251,0.619832,10.239524,24.115354,2892.850131,1946.893513,5606.233351,-86.123272,-0.053128,10359.800596
Bebidas,7.076667,1.334759,7.891571,3.975664,1.593637,7.267027,0.018372,14.590788,237.829404,0.119059,...,4.002040,3.061830,38.650340,28.320470,470.201754,948.710743,2925.771956,210.066785,-33.338153,4521.413084


Las matrices obtenidas son de 48x46

Una vez que ya limpiamos las matrices, las convertimos a formato csv

In [6]:
import os

In [7]:
#La declaración try sirve para evitar error si la carpeta ya existe
try:
    os.mkdir('01 - Matrices de insumo producto')
    
    for pais in paises:
    matrices_IP[pais].to_csv(r'01 - Matrices de insumo producto/'+pais+'_matriz_IP.csv')
    
except:
    print('La carpeta ya existe')

# Vectores económicos

Ahora se obtendrán los vectores que contienen las principales variables económicas de las matrices de insumo-producto

In [8]:
#Para eso, necesitamos una funcion
def creacion_vectores(matriz_IP):
    
    vectores = pd.DataFrame({'Demanda Intermedia': [], 
                        'Demanda Final'     : [],
                        'Producción Final'  : [],
                        'Importaciones'     : [],
                        'Exportaciones'     : [],
                        'Valor Agregado'    : []})
    

    vectores['Demanda Intermedia'] = matriz_IP['DI'][:40]
    
    vectores['Demanda Final']      = (matriz_IP['X']+matriz_IP['Consumo']+matriz_IP['FBK']\
                                      +matriz_IP['Ajuste Preliminar'])[:40]
    vectores['Producción Final']   = matriz_IP['DT a PB'][:40]
    
    vectores['Importaciones']      = (matriz_IP.loc['Importaciones FOB Socios Matriz']\
                                      + matriz_IP.loc['Importaciones FOB RdM'])[:40]
    vectores['Exportaciones']      = matriz_IP['X'][:40]
    
    vectores['Valor Agregado']     = matriz_IP.loc['VAB a PB'][:40]
    
    return vectores

In [9]:
#Guardamos los vectores economicos en diccionario 
vectores_economicos = dict((pais, creacion_vectores(matrices_IP[pais])) for pais in paises)

In [10]:
#Ejemplo con los vectores economicos de Brasil
vectores_economicos['Brasil']

,Demanda Intermedia,Demanda Final,Producción Final,Importaciones,Exportaciones,Valor Agregado
Sector,,,,,,
Agricultura y forestal,50084.704523,22333.631404,72418.335927,933.350337,7492.046275,41730.461105
Caza y pesca,321.639823,909.464303,1231.104125,11.536991,32.051984,643.879354
Minería (energía),24280.727118,4814.739795,29095.466913,2258.169367,3590.276247,12980.149078
Minería (no energía),8063.177728,7504.929692,15568.107420,503.961688,7533.252540,6871.412872
Carne y derivados,5387.364395,21562.602396,26949.966792,598.209611,7190.659052,6991.550415
"Molinería, panadería y pastas",1566.299732,1250.410716,2816.710448,66.030588,4.189803,648.915434
Azúcar y productos de confitería,4137.775279,5894.265736,10032.041014,230.692677,3754.060499,2404.786438
Otros productos alimenticios,20369.700760,38402.482823,58772.183583,1355.868816,7527.526996,14669.426971
Bebidas,6910.121608,4160.778780,11070.900388,258.698558,211.642740,2587.378265


In [11]:
#La declaración try sirve para evitar error si la carpeta ya existe
try:
    os.mkdir('02 - Vectores economicos')
    
    for pais in paises:
        vectores_economicos[pais].to_csv(r'02 - Vectores economicos/'+pais+\
                                        '_vectores_economicos.csv')
    
except:
    print('La carpeta con las archivos ya existe')

# Matriz Regional

## Extración de la la Matriz Regional

Ahora utilizaremos la matriz regional que engloba la información de todos los paises

Esta la podemos extraer por dos vías, directa del repositorio de la CEPAL o en la carpteta local

### Opción 1 - Directo del repositorio de la CEPAL

In [12]:
matriz_regional = pd.read_excel( \
                'https://repositorio.cepal.org/bitstream/handle/11362/40271/15/MIPSudamericana10v1.xlsx',\
                skiprows=[0,1,2,3],nrows=810,usecols=range(3,435))
matriz_regional.columns= map(str, range(432))

### Opción 2 - Usando el directorio local

In [ ]:
matriz_regional = pd.read_excel(
    'Introduzca aquí dirrección local/.../00 - Datos de la CEPAL/MIPSudamericana10v1.xlsx',skiprows=[0,1,2,3],
    nrows=810, usecols=range(3,435))

matriz_regional.columns= map(str, range(432))

In [14]:
#Matriz IP de Sudamérica

matriz_regional

,0,1,2,3,4,5,6,7,8,9,...,422,423,424,425,426,427,428,429,430,431
0,3711.989286,1.558532,0.052407,0.000000,4397.870550,890.470162,189.004489,6928.761143,472.017871,192.151059,...,1.835447,2.559392,145.551895,2.009248e+01,7.064310e+02,1222.264526,385.919983,1183.641602,-685.658705,24128.583711
1,0.050614,30.698698,0.000000,0.000000,4.545148,0.000000,0.000000,105.687518,0.000000,0.000000,...,0.000000,0.000000,18.547199,4.245901e-05,1.867917e+02,0.000000,9.394809,26.382229,2.957677,785.400252
2,0.131935,0.001526,385.871634,0.000000,0.000000,0.000000,0.000000,0.346027,0.000000,0.000000,...,39.025646,0.000000,904.690796,6.974838e-09,6.925017e-07,188.686661,0.000000,0.000000,1746.382667,10999.760305
3,0.215386,0.001745,53.519540,20.403477,0.056237,0.200727,0.278374,0.350668,1.811232,0.000000,...,0.000000,0.000000,0.618761,1.693849e+01,6.218877e+02,158.320709,251.674591,82.231979,8.898145,1923.833009
4,4.349019,0.000000,0.000000,0.000000,344.558172,58.124302,1.529932,18.577375,0.000000,0.000000,...,1.158505,0.000000,79.100708,4.677967e+00,5.447911e+02,10.749461,47.743031,574.839661,-14.142803,6022.496076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,14753.598812,494.121771,7983.450190,1384.212162,409.163555,1456.291178,551.788094,4861.686707,1176.211691,786.778396,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
806,1945.649658,197.315994,557.521118,220.734482,122.503029,476.546112,208.268784,1065.406250,317.389526,445.975403,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
807,12775.503906,295.973999,7407.066406,1158.856079,284.690460,976.792969,341.829956,3779.627686,845.464905,339.342072,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
808,32.445248,0.831778,18.862665,4.621601,1.970066,2.952097,1.689354,16.652771,13.357260,1.460922,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Como vemos tenemos una matriz de 810x432 en la cual, están desglosados todos los datos de los 10 paises, sin embargo, el objetivo es obtener una matriz de 49x46 ya que el interés de esta investigación es tratar a todos estos 10 paises como si fueran uno solo

## Obtención de la matriz regional reducida

El procedimiento para obtener la matriz es el siguiente

In [15]:
sectores = list(matrices_IP['Argentina'].columns[:40])
sectores

['Agricultura y forestal',
 'Caza y pesca',
 'Minería (energía)',
 'Minería (no energía)',
 'Carne y derivados',
 'Molinería, panadería y pastas',
 'Azúcar y productos de confitería',
 'Otros productos alimenticios',
 'Bebidas',
 'Productos de tabaco',
 'Textiles',
 'Confecciones',
 'Calzado',
 'Madera y productos de madera y corcho',
 'Pulpa de madera, papel, imprentas y editoriales',
 'Coque, petróleo refinado y combustible nuclear',
 'Productos químicos básicos',
 'Otros productos químicos (excluye farmacéuticos)',
 'Productos farmacéuticos',
 'Productos de caucho y plástico',
 'Productos minerales no metálicos',
 'Hierro y acero',
 'Metales no ferrosos',
 'Productos fabricados de metal (excepto maquinarias y equipos)',
 'Maquinarias y equipos (excluye maquinaria eléctrica)',
 'Equipos de oficina (incluye equipo computacional)',
 'Maquinarias y aparatos eléctricos',
 'Radio, televisión y equipos de telecomunicaciones',
 'Equipo médico e instrumentos ópticos y de precisión',
 'Vehícu

In [16]:
import numpy as np

#Demanda Interna
ceros = np.zeros((40,40))
for i in range(40):
    filas =matriz_regional.iloc[i:i+361:40].sum()
    for j in range(40):
        columnas = filas[range(j,361+j,40)].sum()
        
        ceros[i,j] = columnas
    
matriz_regional_reducida = pd.DataFrame(ceros, columns=sectores, index=sectores)

#Vector demanda Intermedia
matriz_regional_reducida['DI'] =matriz_regional_reducida.sum(axis=1, numeric_only=True)

#Exportaciones
matriz_regional_reducida['X']= [matriz_regional.iloc[i:i+361:40].sum()[420:430].sum() for i in range(40)]

#Consumo
matriz_regional_reducida['Consumo'] = [matriz_regional.iloc[i:i+361:40].sum()[400:420:2].sum() for i in range(40)]

#Formacion Bruta de Capital
matriz_regional_reducida['FBK'] = [matriz_regional.iloc[i:i+361:40].sum()[401:420:2].sum() for i in range(40)]

#Ajuste preliminar
matriz_regional_reducida['Ajuste Preliminar'] = [matriz_regional.iloc[i:i+361:40].sum()[430] for i in range(40)]

#VBP a Ajuste básico
matriz_regional_reducida['DT a PB'] = matriz_regional_reducida\
[['DI', 'X', 'Consumo','FBK', 'Ajuste Preliminar']].sum(axis=1, numeric_only=True)


#Usos Nacionales a Precios Básicos
matriz_regional_reducida.loc['Usos Nacionales a Precios Básicos'] = \
matriz_regional_reducida.sum(axis=0, numeric_only=True)

matriz_regional_reducida.loc['Usos Nacionales a Precios Básicos'][40:] = np.nan

#Importaciones
ceros = np.zeros((40,40))
for i in range(40):
    sum_filas =matriz_regional.iloc[i+400:i+761:40].sum()
    for j in range(40):
        sum_columnas = sum_filas[range(j,361+j,40)].sum()
        
        ceros[i,j] = sum_columnas
    
matriz_regional_reducida.loc['Importaciones'] = list(ceros.sum(axis=0))+ [np.nan]*6

#Fletes y Seguros
matriz_regional_reducida.loc['Fletes y Seguros']=\
[matriz_regional.iloc[802][i:i+361:40].sum() for i in range(40)]+[np.nan]*6

#Impuestos sobre produccion nacional
matriz_regional_reducida.loc['Impuesto sobre producción nacional']=\
[matriz_regional.iloc[801][i:i+361:40].sum() for i in range(40)]+[np.nan]*6

#Uso al precio de comprador
matriz_regional_reducida.loc['Usos al precio de) comprador'] =\
matriz_regional_reducida.iloc[40]+ matriz_regional_reducida.iloc[41]+\
matriz_regional_reducida.iloc[42]+ matriz_regional_reducida.iloc[43] 

#Remuneraciones
matriz_regional_reducida.loc['Remuneraciones'] = \
[matriz_regional.iloc[806][i:i+361:40].sum() for i in range(40)] + [np.nan]*6

#Excedente bruto de explotacion
matriz_regional_reducida.loc['Excedente Bruto de Explotación'] = \
[matriz_regional.iloc[807][i:i+361:40].sum() for i in range(40)] + [np.nan]*6

#Impuestos menos subsidios
matriz_regional_reducida.loc['Impuesto menos subsidios'] = \
[matriz_regional.iloc[808][i:i+361:40].sum() for i in range(40)] + [np.nan]*6

#VBP a precios básicos
matriz_regional_reducida.loc['VBP a precios básicos'] = \
matriz_regional_reducida.iloc[44]+ matriz_regional_reducida.iloc[45]+\
matriz_regional_reducida.iloc[46]+ matriz_regional_reducida.iloc[47]

matriz_regional_reducida.index.rename('Sector', inplace =True)

In [17]:
#Matriz Regional Reducida de Sudamérica
matriz_regional_reducida

,Agricultura y forestal,Caza y pesca,Minería (energía),Minería (no energía),Carne y derivados,"Molinería, panadería y pastas",Azúcar y productos de confitería,Otros productos alimenticios,Bebidas,Productos de tabaco,...,Correo y telecomunicaciones,Finanzas y seguros,Servicios a empresas de todo tipo,Otros servicios,DI,X,Consumo,FBK,Ajuste Preliminar,DT a PB
Sector,,,,,,,,,,,,,,,,,,,,,
Agricultura y forestal,15233.355278,149.122579,4.816461,31.707231,22520.898705,5750.200803,4661.524855,33288.696153,5053.151685,2024.024805,...,0.659803,1.999616,463.735577,2095.541568,101350.332578,17262.899672,25272.235130,8521.146346,-1137.081259,151269.532467
Caza y pesca,0.707043,183.899716,0.500595,0.270949,70.209068,6.786434,22.331394,2498.951492,26.677162,0.000003,...,0.299120,0.217786,79.279603,208.884811,3169.531497,1655.222206,2911.921026,42.433513,-1.122288,7777.985954
Minería (energía),38.355684,10.756637,6812.465547,414.602499,31.811622,111.034933,63.081747,579.493875,88.645111,1.096020,...,38.077881,35.159951,337.668085,336.756626,59984.266165,51396.506762,641.813948,1142.071711,5643.529074,118808.187660
Minería (no energía),519.213955,22.537011,173.202186,2827.372252,40.520673,20.430399,5.050290,101.528685,11.165324,1.063849,...,4.618001,65.211434,113.364727,121.254112,20120.806277,35458.774291,203.479608,961.465907,1445.153534,58189.679616
Carne y derivados,25.256159,24.514295,2.127941,16.483941,2161.293782,229.537375,214.652565,1436.143769,242.952110,0.086526,...,1.694442,2.698801,840.629352,3182.437845,10737.244780,9277.933881,29668.906191,-38.888996,-57.145329,49588.050528
"Molinería, panadería y pastas",288.967001,12.296842,8.022588,3.885161,364.839577,3075.118957,167.431800,1201.973986,344.954532,5.281723,...,4.340464,7.108278,130.843340,1829.555577,7719.507914,492.352620,17366.685396,16.973276,77.615250,25673.134457
Azúcar y productos de confitería,23.278196,2.059435,2.104847,8.372521,786.904888,493.856245,701.634767,2057.535841,807.839896,20.669564,...,5.221632,11.644490,270.551790,538.294846,6323.377786,4224.299498,5089.776720,-158.861801,-83.967787,15394.624414
Otros productos alimenticios,9122.877211,995.759379,38.089468,35.794426,3154.954599,1653.750092,1098.843243,10289.376879,1676.067874,2.294724,...,10.011548,28.612504,1561.087712,4283.608046,35427.540671,22993.941513,51685.808483,289.751022,-738.066898,109658.974791
Bebidas,26.673240,3.939464,10.689853,8.424098,164.819770,39.195166,70.078763,409.234690,1155.195740,3.991561,...,9.691870,10.396379,6112.342901,3761.030040,12154.913895,1402.663007,15179.473718,345.473986,-305.678332,28776.846274


### Vectores económicos regionales

También obtenemos los vectores económicos regionales

In [18]:
vectores_eco_regionales = pd.DataFrame({'Demanda Intermedia': [], 
                        'Demanda Final'     : [],
                        'Producción Final'  : [],
                        'Importaciones'     : [],
                        'Exportaciones'     : [],
                        'Valor Agregado'    : []})
    

vectores_eco_regionales['Demanda Intermedia'] = matriz_regional_reducida['DI'][:40]

vectores_eco_regionales['Demanda Final']      = (matriz_regional_reducida['X']+matriz_regional_reducida['Consumo']+\
                                                 matriz_regional_reducida['FBK']\
                                                 +matriz_regional_reducida['Ajuste Preliminar'])[:40]

vectores_eco_regionales['Producción Final']   = matriz_regional_reducida['DT a PB'][:40]

vectores_eco_regionales['Importaciones']      = matriz_regional_reducida.loc['Importaciones'][:40]

vectores_eco_regionales['Exportaciones']      = matriz_regional_reducida['X'][:40]

vectores_eco_regionales['Valor Agregado']     = matriz_regional_reducida.loc['Remuneraciones'][:40]+\
                                                matriz_regional_reducida.loc['Excedente Bruto de Explotación'][:40]+\
                                                matriz_regional_reducida.loc['Impuesto menos subsidios'][:40] 

In [19]:
vectores_eco_regionales

,Demanda Intermedia,Demanda Final,Producción Final,Importaciones,Exportaciones,Valor Agregado
Sector,,,,,,
Agricultura y forestal,101350.332578,49919.199888,151269.532467,2978.744851,17262.899672,88322.826013
Caza y pesca,3169.531497,4608.454457,7777.985954,171.523136,1655.222206,4200.211535
Minería (energía),59984.266165,58823.921495,118808.187660,6086.636626,51396.506762,78145.682735
Minería (no energía),20120.806277,38068.873339,58189.679616,1118.429472,35458.774291,37763.667021
Carne y derivados,10737.244780,38850.805748,49588.050528,637.859397,9277.933881,10022.331798
"Molinería, panadería y pastas",7719.507914,17953.626543,25673.134457,1091.344005,492.352620,6918.974371
Azúcar y productos de confitería,6323.377786,9071.246629,15394.624414,292.175291,4224.299498,4172.571376
Otros productos alimenticios,35427.540671,74231.434121,109658.974791,2038.306963,22993.941513,28347.508919
Bebidas,12154.913895,16621.932379,28776.846274,738.785536,1402.663007,8811.383956


### Por último guardamos los datos en archivos .csv

In [20]:
matriz_regional_reducida.to_csv(r'01 - Matrices de insumo producto/Sudamerica_matriz_IP.csv')
vectores_eco_regionales.to_csv(r'02 - Vectores economicos/Sudamerica_vectores_economicos.csv')

----

# Información del notebook

In [21]:
import session_info
session_info.show(cpu=True, html=False)

-----
numpy               1.21.2
pandas              1.3.3
session_info        1.0.0
-----
IPython             7.20.0
jupyter_client      6.1.11
jupyter_core        4.7.1
notebook            6.2.0
-----
Python 3.8.10 (default, Mar 13 2023, 10:26:41) [GCC 9.4.0]
Linux-5.15.0-67-generic-x86_64-with-glibc2.29
4 logical CPU cores, x86_64
-----
Session information updated at 2023-03-25 20:43
